In [105]:
import pandas as pd
import numpy as np
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from math import sqrt

In [106]:
dataset=pd.read_csv("Dataset/DataCoSupplyChainDataset.csv",header= 0,encoding= 'unicode_escape')
dataset.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [107]:
dataset.shape

(180519, 53)

In [108]:
data=dataset.drop(columns=['Customer Fname','Customer Lname','Order Zipcode', 'Product Description','Customer Email','Customer Password','Customer Street','Latitude','Longitude','Order Id','Order Item Cardprod Id','Order Zipcode','Product Description','Product Image'])
data=data.drop(columns=['Category Name', 'Customer City','Customer Country', 'Customer Id', 'Customer Segment', 'Customer State','Order City', 'Order Country','Order Country'])
data['Customer Zipcode']=data['Customer Zipcode'].fillna(0)
# dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str)+dataset['Customer Lname'].astype(str)
# data=dataset.drop(columns=['Customer Fname','Customer Lname','Order Zipcode', 'Product Description','Customer Email','Customer Password','Customer Street','Latitude','Longitude','Order Id','Order Item Cardprod Id','Order Zipcode','Product Description','Product Image'])

In [109]:
data.shape

(180519, 33)

In [110]:
data.isnull().sum()

Type                             0
Days for shipping (real)         0
Days for shipment (scheduled)    0
Benefit per order                0
Sales per customer               0
Delivery Status                  0
Late_delivery_risk               0
Category Id                      0
Customer Zipcode                 0
Department Id                    0
Department Name                  0
Market                           0
Order Customer Id                0
order date (DateOrders)          0
Order Item Discount              0
Order Item Discount Rate         0
Order Item Id                    0
Order Item Product Price         0
Order Item Profit Ratio          0
Order Item Quantity              0
Sales                            0
Order Item Total                 0
Order Profit Per Order           0
Order Region                     0
Order State                      0
Order Status                     0
Product Card Id                  0
Product Category Id              0
Product Name        

In [111]:
data = data.dropna()
data.isnull().sum()
data.shape

(180519, 33)

In [112]:
data=data.rename(columns = {'order date (DateOrders)': 'Order_Date'}, inplace = False)

In [113]:
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
               # deleting the column from the dataset
    return(list(col_corr))
cols=correlation(data,0.5)

In [114]:
print(cols)

['Order Item Discount Rate', 'Order Item Product Price', 'Order Profit Per Order', 'Order Item Profit Ratio', 'Order Item Total', 'Product Category Id', 'Sales', 'Product Price', 'Days for shipment (scheduled)', 'Product Card Id', 'Department Id']


In [115]:
#new=data.loc[data['Department Id'] == 2]
final_cols=['Order_Date','Order Item Quantity','Department Id','Product Category Id', 'Order Customer Id',  'Days for shipment (scheduled)', 'Order Item Profit Ratio', 'Product Card Id', 'Sales', 'Order Item Discount Rate', 'Order Item Product Price']

In [116]:
data=data[final_cols]

In [117]:
data.head()

,Order_Date,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Days for shipment (scheduled),Order Item Profit Ratio,Product Card Id,Sales,Order Item Discount Rate,Order Item Product Price
0,1/31/2018 22:56,1,2,73,20755,4,0.29,1360,327.75,0.04,327.75
1,1/13/2018 12:27,1,2,73,19492,4,-0.80,1360,327.75,0.05,327.75
2,1/13/2018 12:06,1,2,73,19491,4,-0.80,1360,327.75,0.06,327.75
3,1/13/2018 11:45,1,2,73,19490,4,0.08,1360,327.75,0.07,327.75
4,1/13/2018 11:24,1,2,73,19489,4,0.45,1360,327.75,0.09,327.75


In [118]:
numerical_features = [f for f in data.columns if data[f].dtypes!='O']
only=data[numerical_features]
only.head()

,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Days for shipment (scheduled),Order Item Profit Ratio,Product Card Id,Sales,Order Item Discount Rate,Order Item Product Price
0,1,2,73,20755,4,0.29,1360,327.75,0.04,327.75
1,1,2,73,19492,4,-0.80,1360,327.75,0.05,327.75
2,1,2,73,19491,4,-0.80,1360,327.75,0.06,327.75
3,1,2,73,19490,4,0.08,1360,327.75,0.07,327.75
4,1,2,73,19489,4,0.45,1360,327.75,0.09,327.75


In [119]:
only=only.drop(columns=['Department Id','Order Item Quantity','Product Category Id','Order Customer Id','Product Card Id'])
only_features = [f for f in only.columns ]
values = only.values
values = values.reshape((len(values), len(only_features)))
only.head()

,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,4,0.29,327.75,0.04,327.75
1,4,-0.80,327.75,0.05,327.75
2,4,-0.80,327.75,0.06,327.75
3,4,0.08,327.75,0.07,327.75
4,4,0.45,327.75,0.09,327.75


In [120]:
# # train the standardization
scaler = StandardScaler()
scaler = scaler.fit(values)

# # standardization the dataset and print the first 5 rows
normalized = scaler.transform(values)
for i in range(5):
	print(normalized[i])

[ 0.77715232  0.36280081  0.93729011 -0.87578178  1.33482173]
[ 0.77715232 -1.97227478  0.93729011 -0.73376659  1.33482173]
[ 0.77715232 -1.97227478  0.93729011 -0.59175147  1.33482173]
[ 0.77715232 -0.08707612  0.93729011 -0.4497363   1.33482173]
[ 0.77715232  0.70556419  0.93729011 -0.16570593  1.33482173]


In [121]:
# # inverse transform and print the first 5 rows
# inversed = scaler.inverse_transform(normalized)
# for i in range(5):
# 	print(inversed[i])

In [122]:
x=pd.DataFrame(data=normalized,index=range(0,len(only)),columns=only_features)
x.head()

,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,0.777152,0.362801,0.93729,-0.875782,1.334822
1,0.777152,-1.972275,0.93729,-0.733767,1.334822
2,0.777152,-1.972275,0.93729,-0.591751,1.334822
3,0.777152,-0.087076,0.93729,-0.449736,1.334822
4,0.777152,0.705564,0.93729,-0.165706,1.334822


In [123]:
data.drop(labels=x.columns, axis="columns", inplace=True)
data[x.columns] = x[x.columns]
data.head()

,Order_Date,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Product Card Id,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,1/31/2018 22:56,1,2,73,20755,1360,0.777152,0.362801,0.93729,-0.875782,1.334822
1,1/13/2018 12:27,1,2,73,19492,1360,0.777152,-1.972275,0.93729,-0.733767,1.334822
2,1/13/2018 12:06,1,2,73,19491,1360,0.777152,-1.972275,0.93729,-0.591751,1.334822
3,1/13/2018 11:45,1,2,73,19490,1360,0.777152,-0.087076,0.93729,-0.449736,1.334822
4,1/13/2018 11:24,1,2,73,19489,1360,0.777152,0.705564,0.93729,-0.165706,1.334822


In [124]:
#data=data.set_index('Order_Date')
data.Order_Date = pd.DatetimeIndex(data.Order_Date).to_period('D')
#data.sort_index(inplace=True)
#data=data.sort_values(by = 'Order_Date') 

In [125]:
data.head()

,Order_Date,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Product Card Id,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,2018-01-31,1,2,73,20755,1360,0.777152,0.362801,0.93729,-0.875782,1.334822
1,2018-01-13,1,2,73,19492,1360,0.777152,-1.972275,0.93729,-0.733767,1.334822
2,2018-01-13,1,2,73,19491,1360,0.777152,-1.972275,0.93729,-0.591751,1.334822
3,2018-01-13,1,2,73,19490,1360,0.777152,-0.087076,0.93729,-0.449736,1.334822
4,2018-01-13,1,2,73,19489,1360,0.777152,0.705564,0.93729,-0.165706,1.334822


In [126]:
data.shape

(180519, 11)

In [127]:
data=data.dropna()

In [128]:
data.isnull().sum()

Order_Date                       0
Order Item Quantity              0
Department Id                    0
Product Category Id              0
Order Customer Id                0
Product Card Id                  0
Days for shipment (scheduled)    0
Order Item Profit Ratio          0
Sales                            0
Order Item Discount Rate         0
Order Item Product Price         0
dtype: int64

In [129]:
data.to_csv('Dataset/Cleaned_data.csv',  header=True,index = False)